<a href="https://colab.research.google.com/github/Ben-Najafloo/Fairness-evaluation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install aif360
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric


In [4]:
# Step 1: Dataset Fairness Evaluation
def assess_dataset_fairness(data, sensitive_attribute, label_column):
    privileged_group = data[data[sensitive_attribute] == 1]
    unprivileged_group = data[data[sensitive_attribute] == 0]

    overall_positive_rate = data[label_column].mean()
    privileged_rate = privileged_group[label_column].mean()
    unprivileged_rate = unprivileged_group[label_column].mean()

    # Fairness metrics
    statistical_parity_difference = unprivileged_rate - privileged_rate
    disparate_impact = (
        unprivileged_rate / privileged_rate if privileged_rate > 0 else 0
    )

    return {
        "Overall Positive Rate": overall_positive_rate,
        "Privileged Group Positive Rate": privileged_rate,
        "Unprivileged Group Positive Rate": unprivileged_rate,
        "Statistical Parity Difference": statistical_parity_difference,
        "Disparate Impact": disparate_impact,
    }

In [5]:
# Preprocessing for AIF360
def preprocess_data(data, sensitive_attribute, label_column):
    dataset = StandardDataset(
        df=data,
        label_name=label_column,
        favorable_classes=[1],
        protected_attribute_names=[sensitive_attribute],
        privileged_classes=[[1]]
    )
    return dataset

In [18]:
def train_and_evaluate_fairness(data, sensitive_attribute, label_column):
    try:
        # Split dataset into train and test sets
        train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

        # Prepare train and test sets for fairness evaluation
        privileged_groups = [{sensitive_attribute: 1}]
        unprivileged_groups = [{sensitive_attribute: 0}]
        dataset_train = StandardDataset(
            train_data, label_name=label_column, favorable_classes=[1],
            protected_attribute_names=[sensitive_attribute],
            privileged_classes=[[1]]
        )
        dataset_test = StandardDataset(
            test_data, label_name=label_column, favorable_classes=[1],
            protected_attribute_names=[sensitive_attribute],
            privileged_classes=[[1]]
        )

        # Train a simple logistic regression model
        X_train = pd.DataFrame(dataset_train.features, columns=dataset_train.feature_names)
        y_train = dataset_train.labels.ravel()
        X_test = pd.DataFrame(dataset_test.features, columns=dataset_test.feature_names)
        y_test = dataset_test.labels.ravel()
        model = LogisticRegression(solver='liblinear').fit(X_train, y_train)

        # Predict on test data
        predictions = model.predict(X_test)
        dataset_test_pred = dataset_test.copy()
        dataset_test_pred.labels = predictions

        # Compute fairness metrics
        classification_metric = ClassificationMetric(
            dataset_test, dataset_test_pred,
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups
        )

        fairness_metrics = {
            "Accuracy": model.score(X_test, y_test),
            "Equal Opportunity Difference": classification_metric.equal_opportunity_difference(),
            "Average Odds Difference": classification_metric.average_odds_difference(),
            "Disparate Impact": classification_metric.disparate_impact(),
        }

        # Handle nan values in metrics
        for metric, value in fairness_metrics.items():
            if isinstance(value, float) and np.isnan(value):
                fairness_metrics[metric] = 0.0  # Replace nan with 0.0

        return fairness_metrics

    except Exception as e:
        print(f"Error occurred during model training or fairness evaluation: {e}")
        # Return default fairness metrics in case of error
        return {
            "Accuracy": 0.0,
            "Equal Opportunity Difference": 0.0,
            "Average Odds Difference": 0.0,
            "Disparate Impact": 0.0,
        }


In [22]:
# Main Workflow
def main():
    # Example dataset
    data = pd.DataFrame({
        'age': [25, 45, 35, 50, 23, 40, 32, 47, 35, 50, 23, 40, 32, 47, 35, 50, 23, 40, 32, 47],
        'income': [50000, 80000, 62000, 72000, 52000, 68000, 59000, 77000, 80000, 62000, 72000, 52000, 80000, 62000, 72000, 52000, 80000, 62000, 72000, 52000],
        'gender': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],  # 0: Female, 1: Male
        'loan_approved': [1, 0, 1, 1,1, 0, 1, 1,1, 0, 1, 1,1, 0, 1, 1, 0, 0, 1, 0]
    })

    # Sensitive attribute and label column
    sensitive_attribute = 'gender'
    label_column = 'loan_approved'

    # Step 1: Evaluate dataset fairness
    dataset_fairness = assess_dataset_fairness(data, sensitive_attribute, label_column)
    print("\n=== Dataset Fairness Metrics ===")
    for metric, value in dataset_fairness.items():
        print(f"{metric}: {value:.4f}")

    # Ask user if they want to proceed
    proceed = input("\nDo you want to evaluate fairness on the trained model? (yes/no): ").strip().lower()
    if proceed != 'yes':
        print("Exiting the process.")
        return

    # Step 2: Train model and evaluate fairness
    model_fairness = train_and_evaluate_fairness(data, sensitive_attribute, label_column)
    print("\n=== Model Fairness Metrics ===")
    for metric, value in model_fairness.items():
        print(f"{metric}: {value:.4f}")

if __name__ == "__main__":
    main()


=== Dataset Fairness Metrics ===
Overall Positive Rate: 0.6500
Privileged Group Positive Rate: 0.4000
Unprivileged Group Positive Rate: 0.9000
Statistical Parity Difference: 0.5000
Disparate Impact: 2.2500

Do you want to evaluate fairness on the trained model? (yes/no): yes

=== Model Fairness Metrics ===
Accuracy: 0.2500
Equal Opportunity Difference: 1.0000
Average Odds Difference: 0.0000
Disparate Impact: 1.5000


/usr/local/lib/python3.10/dist-packages/aif360/metrics/classification_metric.py:278: RuntimeWarning: invalid value encountered in scalar divide
  TPR=TP / P, TNR=TN / N, FPR=FP / N, FNR=FN / P,
/usr/local/lib/python3.10/dist-packages/aif360/metrics/classification_metric.py:279: RuntimeWarning: invalid value encountered in scalar divide
  GTPR=GTP / P, GTNR=GTN / N, GFPR=GFP / N, GFNR=GFN / P,
